# Optimization schemes
In this notebook we work on implementing different optimization methods and applying them to the minimization of several bivariate functions. 

The (python) functions that need to be implemented are the first ones, in order to be able to compute the gradient of a two-dimensional function and to implement the methods of gradient descent with and without momentum, and Nesterov accelerated gradient. In the rest of the notebook, those methods are applied to different functions, from the “simplest” to the most complex. Interactive representations make it possible to visualize the behavior of the algorithms as the step, initial conditions, and number of iterations change.

The imports are classical, with the addition of some plotting and error computation functions that are needed to visualize the algorithms (and which are in the file `lab2_auxiliary.py`).

In [ ]:
import torch 
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact, fixed, IntSlider, FloatSlider
import numpy as np

from lab2_auxiliary import plot_3D, plot_3D_GD, default_pars, err

## Computing the gradient of a two dimensional function
In the following function, given a function $f:\mathbb{R}^2\to \mathbb{R}$ and a set of points $(x_i, y_i)_{i=1}^N$, we want to compute
$$
    (\nabla f)(x_i , y_i), \qquad i=1, \dots, N
$$
using automatic differentiation. We will use the function `torch.autograd.grad` and a trick, that we will explain in one dimention. Let $g:\mathbb{R}\to\mathbb{R}$. By passing $x_i$ and $g(x_i)$ to `torch.autograd.grad`, we can compute the derivative $g'(x_i) = (\partial_{x_i} g)(x_i)$; in order to compute the derivative at all $(x_i)_{i=1}^N$, we can exploit the fact that
$$
\partial_{x_i}  \left(\sum_{i=1}^N g(x_i) \right)= g'(x_i).
$$
In other words, if we feed  `torch.autograd.grad` the vector $(x_i)_{i=1}^N $ and the scalar $\sum_{i=1}^N g(x_i)$, we obtain what we wanted.

In [ ]:
def grad(x,y,fun):
    x.requires_grad_(True)
    y.requires_grad_(True)
    u_x = torch.autograd.grad(fun(x,y).sum(),x,create_graph=True)[0]
    u_y = torch.autograd.grad(fun(x,y).sum(),y,create_graph=True)[0]
    return torch.Tensor([u_x,u_y])



## Gradient descent
Complete the following function to implement the gradient descent method: given an initial guess $z_0 = (x_0, y_0)\in \mathbb{R}^2$ and a vector $(\eta_k)_{k=1}^{N_{\mathrm{epochs}}}$, for $k=1, \dots, N_{\mathrm{epochs}}$ compute
$$
z_k = z_{k-1} - \eta_k \nabla f(z_{k-1}).
$$
If $\eta_k \equiv \eta \in \mathbb{R}_+$ for all $k$, the function takes also a scalar $\eta$ as input.

In [ ]:
def GD(x_0,y_0,eta,num_epoch,fun):
    X = torch.empty([num_epoch,2])
    X[0,:] = torch.Tensor([x_0,y_0]) # a torch.Tensor containing the initial guess

    if np.size(eta) == 1: 
        for i in range(1,num_epoch):
            # a for cycle, and the computation of X[k, :] from X[k-1, :]
            X[i,:] = X[i-1,:] - eta*grad(X[i-1,0],X[i-1,1],fun)
    else:
        assert np.size(eta) == num_epoch
        for i in range(1,num_epoch):
            # similar to above
            X[i,:] = X[i-1,:] - eta[i-1]*grad(X[i-1,0],X[i-1,1],fun)

    return X



## Momentum
We want to implement here a gradient descent method with momentum. Given $x_0$ and $v_0$ in $\mathbb{R}^2$ and $\alpha, \beta\in \mathbb{R}_+$, we compute for $k=1, \dots, N_{\mathrm{epochs}}$
$$
      \begin{aligned}
        v_{k} &= \beta v_{k-1}  -\alpha \nabla f(x_{k-1}),\\
        x_{k} &= x_{k-1} + v_{k}.
      \end{aligned}
$$

In [ ]:
def momentum(x_0,y_0,v,beta,eta,num_epochs,fun):
    # args: x_0 and y_0 are scalars, v is in R^2 instead
    X = torch.empty() # store all values here (we want it for the plots)
    V = torch.empty() # here we want to store only v_k and v_{k-1}
    X[0,:] = 
    V[0,:] = 
    # for cycle here implementing the algorithm
    

    return X



## Nesterov's method
We conclude with Nesterov's method. Given $x_0$ and $v_0$ in $\mathbb{R}^2$ and $\alpha, \beta\in \mathbb{R}_+$, we compute for $k=1, \dots, N_{\mathrm{epochs}}$
$$
      \begin{aligned}
        v_{k} &= \beta v_{k-1}  -\alpha \nabla f(x_{k-1} + \beta v_{k-1}),\\
        x_{k} &= x_{k-1} + v_{k}.
      \end{aligned}
$$## Nesterov's method

In [ ]:

def Nesterov(x_0,y_0,v,beta,eta,num_epochs,fun):
    X = 
    V = 
    
    # initialize X_0 V_0

    # loop 

    return X

The following function is necessary for the interactive plots in the next sections (and is already complete) 

In [ ]:
def plot_2D_GD_interactive(method, x, y, x_min, y_min, fun, x_0, y_0, eta, num_epochs):
    params = default_pars()
    params['x_0'] = x_0
    params['y_0'] = y_0
    params['learning_rate'] = eta
    params['num_epochs'] = num_epochs
    if method == 'gd':
        update = GD(params['x_0'], params['y_0'], params['learning_rate'], params['num_epochs'], fun)
    elif method == 'momentum':
        update = momentum(params['x_0'], params['y_0'], torch.zeros(2), 0.5, params['learning_rate'], params['num_epochs'], fun)
    elif method == 'nesterov':
        update = Nesterov(params['x_0'], params['y_0'], torch.zeros(2), 0.5, params['learning_rate'], params['num_epochs'], fun)

    x_mesh, y_mesh = torch.meshgrid(x, y, indexing='ij')
    with torch.inference_mode():
        fig, ax = plt.subplots(figsize = (4,4))
        x_plt, y_plt = x_mesh.detach().numpy(), y_mesh.detach().numpy()
        contour = ax.contourf(x_plt, y_plt, fun(x_mesh, y_mesh).detach().numpy())

        ax.plot(update[:, 0], update[:, 1], marker = 'o', c = 'red', linewidth = 0.7, markersize = 3)
        ax.scatter(x_min, y_min, marker = 'X', label = 'global minumum')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.legend()
        ax.set_aspect('equal', adjustable='box')
        fig.colorbar(contour, ax=ax, orientation='vertical', label='Function Value') 

## A convex (quadratic) function
The first example we consider is a simple quadratic function



In [ ]:
def f(x, y):
    return x**2 + y**2

x = torch.linspace(-5, 5, 100).requires_grad_(True)
y = torch.linspace(-5, 5, 100).requires_grad_(True)

fig = plot_3D(x, y, f)
fig.show()

### GD

In [ ]:
x_0, y_0 = 2, 3
learning_rate = 0.1
num_epochs = 10

gd = GD(x_0, y_0, learning_rate, num_epochs, f)

fig_3D = plot_3D_GD(x, y, gd[:,0], gd[:,1], f) 
fig_3D.show()

In [ ]:
_ = interact(plot_2D_GD_interactive, 
            method = fixed('gd'),
             x = fixed(x), 
             y = fixed(y), 
             x_min = fixed(0), 
             y_min = fixed(0), 
             fun = fixed(f), 
             x_0 = FloatSlider(min=-5, max=5, step=0.1, value=2),
             y_0 = FloatSlider(min=-5, max=5, step=0.1, value=3),
             eta = FloatSlider(min = 0, max = 1, step = 0.01, value=0.1), 
             num_epochs = IntSlider(min = 1, max = 100, step = 1, value=10))

### Momentum

In [ ]:
V = torch.zeros(2)
learning_rate = 0.2
num_epochs = 10
beta = 0.5

mom = momentum(x_0, y_0, V, beta, learning_rate, num_epochs, f)

fig_3D = plot_3D_GD(x, y, mom[:,0], mom[:,1], f)
fig_3D.show()

In [ ]:
%matplotlib inline
_ = interact(plot_2D_GD_interactive, 
            method = fixed('momentum'),
             x = fixed(x), 
             y = fixed(y), 
             x_min = fixed(0), 
             y_min = fixed(0), 
             fun = fixed(f), 
             x_0 = FloatSlider(min=-5, max=5, step=0.1, value=2),
             y_0 = FloatSlider(min=-5, max=5, step=0.1, value=3),
             eta = FloatSlider(min = 0, max = 1, step = 0.01, value=0.1), 
             num_epochs = IntSlider(min = 1, max = 100, step = 1, value=10))

### Nesterov


In [ ]:
beta = 0.5
nes = Nesterov(x_0,y_0,V,beta,learning_rate,num_epochs,f)

fig_3D = plot_3D_GD(x, y, nes[:,0], nes[:,1], f)
fig_3D.show()

In [ ]:
%matplotlib inline
_ = interact(plot_2D_GD_interactive, 
            method = fixed('nesterov'),
             x = fixed(x), 
             y = fixed(y), 
             x_min = fixed(0), 
             y_min = fixed(0), 
             fun = fixed(f), 
             x_0 = FloatSlider(min=-5, max=5, step=0.1, value=2),
             y_0 = FloatSlider(min=-5, max=5, step=0.1, value=3),
             eta = FloatSlider(min = 0, max = 1, step = 0.01, value=0.1), 
             num_epochs = IntSlider(min = 1, max = 100, step = 1, value=10))

### Convergence

In [ ]:
min_glob = torch.Tensor([0,0])
dist_err_gd = err(gd,min_glob,f)
dist_err_mom = err(mom,min_glob,f)
dist_err_nes = err(nes,min_glob,f)


plt.semilogy(range(len(gd)),dist_err_gd, marker = 'o', label = 'GD')
plt.semilogy(range(len(mom)),dist_err_mom,marker = 'x', label = 'Momentum')
plt.semilogy(range(len(nes)),dist_err_nes,marker = 'd', label = 'Nesterov')

plt.grid()
plt.xlabel('Epochs')
plt.title('Distance from the global min')
plt.legend()
plt.show()

## A function with a local and a global minimum (and saddle points)

In [ ]:
def g(x,y):
    return 2/5 -(1/10)*(5*x**2 + 5*y**2 + 6*x*y - x - 2*y)*torch.exp(-(x**2+y**2))

x = torch.linspace(-2,2,100).requires_grad_(True)
y = torch.linspace(-2,2,100).requires_grad_(True)

plot_3D(x,y,g)

### GD

In [ ]:
x_0, y_0 = -0, 0
learning_rate = 0.1
num_epochs = 10

gd = GD(x_0,y_0,learning_rate,num_epochs,g)
fig_3D = plot_3D_GD(x, y, gd[:,0], gd[:,1], g)
fig_3D.show()

In [ ]:
%matplotlib inline
_ = interact(plot_2D_GD_interactive, 
            method = fixed('gd'),
             x = fixed(x), 
             y = fixed(y), 
             x_min = fixed(-0.63), 
             y_min = fixed(-0.70), 
             fun = fixed(g), 
             x_0 = FloatSlider(min=-2, max=2, step=0.1, value=0),
             y_0 = FloatSlider(min=-2, max=2, step=0.1, value=0),
             eta = FloatSlider(min = 0, max = 1, step = 0.01, value=0.2), 
             num_epochs = IntSlider(min = 1, max = 100, step = 1, value=10))

### Momentum

In [ ]:
V = torch.zeros(2)
learning_rate = 0.2
num_epochs = 10
beta = 0.5

mom = momentum(x_0,y_0,V,beta,learning_rate,num_epochs,g)

fig_3D = plot_3D_GD(x,y,mom[:,0],mom[:,1],g)
fig_3D.show()


In [ ]:
%matplotlib inline
_ = interact(plot_2D_GD_interactive, 
            method = fixed('momentum'),
             x = fixed(x), 
             y = fixed(y), 
             x_min = fixed(-0.63), 
             y_min = fixed(-0.70), 
             fun = fixed(g), 
             x_0 = FloatSlider(min=-2, max=2, step=0.1, value=0),
             y_0 = FloatSlider(min=-2, max=2, step=0.1, value=0),
             eta = FloatSlider(min = 0, max = 1, step = 0.01, value=0.2), 
             num_epochs = IntSlider(min = 1, max = 100, step = 1, value=10))

### Nesterov

In [ ]:
beta = 0.5
nes = Nesterov(x_0,y_0,V,beta,learning_rate,num_epochs,g)

fig_3D = plot_3D_GD(x,y,nes[:,0],nes[:,1],g)
fig_3D.show()

In [ ]:
%matplotlib inline
_ = interact(plot_2D_GD_interactive, 
            method = fixed('nesterov'),
             x = fixed(x), 
             y = fixed(y), 
             x_min = fixed(-0.63), 
             y_min = fixed(-0.70), 
             fun = fixed(g), 
             x_0 = FloatSlider(min=-2, max=2, step=0.1, value=0),
             y_0 = FloatSlider(min=-2, max=2, step=0.1, value=0),
             eta = FloatSlider(min = 0, max = 1, step = 0.01, value=0.2), 
             num_epochs = IntSlider(min = 1, max = 100, step = 1, value=10))

In [ ]:
min_glob = torch.Tensor([-0.63,-0.7])
dist_err_gd = err(gd,min_glob,g)
dist_err_mom = err(mom,min_glob,g)
dist_err_nes = err(nes,min_glob,g)

plt.semilogy(range(len(gd)),dist_err_gd, marker = 'o', label = 'GD')
plt.semilogy(range(len(mom)),dist_err_mom,marker = 'x', label = 'Momentum')
plt.semilogy(range(len(nes)),dist_err_nes,marker = 'd', label = 'Nesterov')

plt.grid()
plt.xlabel('Epochs')
plt.title('Distance from the global min')
plt.legend()
plt.show()


## An anisotropic function

In [ ]:
def h(x,y):
    return (1-x)**2+100*(y-x**2)**2

x = torch.linspace(-1.5,1.5,100).requires_grad_(True)
y = torch.linspace(-1,3,100).requires_grad_(True)

plot_3D(x,y,h)

### GD

In [ ]:
x_0, y_0 = -1, -0.5
learning_rate = 0.002
num_epochs = 40

gd = GD(x_0,y_0,learning_rate,num_epochs,h)
fig_3D = plot_3D_GD(x,y,gd[:,0],gd[:,1],h)
fig_3D.show()

In [ ]:
%matplotlib inline
_ = interact(plot_2D_GD_interactive, 
            method = fixed('gd'),
             x = fixed(x), 
             y = fixed(y), 
             x_min = fixed(1), 
             y_min = fixed(1), 
             fun = fixed(h), 
             x_0 = FloatSlider(min=-1.5, max=1.5, step=0.1, value=-1),
             y_0 = FloatSlider(min=-1, max=3, step=0.1, value=-0.5),
             eta = FloatSlider(min = 0, max = 0.004, step = 0.0005, value=0.002, readout_format='.4f'), 
             num_epochs = IntSlider(min = 1, max = 100, step = 1, value=10))

### Momentum


In [ ]:
V = torch.zeros(2)
learning_rate = 0.002
num_epochs = 10
beta = 0.5

mom = momentum(x_0,y_0,V,beta,learning_rate,num_epochs,h)

fig_3D = plot_3D_GD(x,y,mom[:,0],mom[:,1],h)
fig_3D.show()

In [ ]:
%matplotlib inline
_ = interact(plot_2D_GD_interactive, 
            method = fixed('momentum'),
             x = fixed(x), 
             y = fixed(y), 
             x_min = fixed(1), 
             y_min = fixed(1), 
             fun = fixed(h), 
             x_0 = FloatSlider(min=-1.5, max=1.5, step=0.1, value=-1),
             y_0 = FloatSlider(min=-1, max=3, step=0.1, value=-0.5),
             eta = FloatSlider(min = 0, max = 0.004, step = 0.0005, value=0.002, readout_format='.4f'), 
             num_epochs = IntSlider(min = 1, max = 100, step = 1, value=10))

### Nesterov

In [ ]:
beta = 0.49
nes = Nesterov(x_0,y_0,V,beta,learning_rate,num_epochs,h)

fig_3D = plot_3D_GD(x,y,nes[:,0],nes[:,1],h)
fig_3D.show()

In [ ]:
%matplotlib inline
_ = interact(plot_2D_GD_interactive, 
            method = fixed('nesterov'),
             x = fixed(x), 
             y = fixed(y), 
             x_min = fixed(1), 
             y_min = fixed(1), 
             fun = fixed(h), 
             x_0 = FloatSlider(min=-1.5, max=1.5, step=0.1, value=-1),
             y_0 = FloatSlider(min=-1, max=3, step=0.1, value=-0.5),
             eta = FloatSlider(min = 0, max = 0.004, step = 0.0005, value=0.002, readout_format='.4f'), 
             num_epochs = IntSlider(min = 1, max = 100, step = 1, value=10))

In [ ]:
min_glob = torch.Tensor([1,1])
dist_err_gd = err(gd,min_glob,h)
dist_err_mom = err(mom,min_glob,h)
dist_err_nes = err(nes,min_glob,h)
plt.semilogy(range(len(gd)),dist_err_gd, marker = 'o', label = 'GD')
plt.semilogy(range(len(mom)),dist_err_mom,marker = 'x', label = 'Momentum')
plt.semilogy(range(len(nes)),dist_err_nes,marker = 'd', label = 'Nesterov')

plt.grid()
plt.xlabel('Epochs')
plt.title('Distance from the global min')
plt.legend()
plt.show()

## A function with many local minima/Importance of the learning rate

In [ ]:
def ackley(x,y):
    return -20*torch.exp(-0.2*torch.sqrt(0.5*(x**2+y**2)))-torch.exp(0.5*(torch.cos(2*torch.pi*x)+torch.cos(2*torch.pi*y)))+torch.exp(torch.Tensor([1]))+20
x = torch.linspace(-4,4,200).requires_grad_(True)
y = torch.linspace(-4,4,200).requires_grad_(True)

plot_3D(x,y,ackley)

### GD

In [ ]:
x_0, y_0 = -2, 3
learning_rate = 0.2
num_epochs = 10 

gd = GD(x_0,y_0,learning_rate,num_epochs,ackley)

plot_3D_GD(x,y,gd[:,0],gd[:,1],ackley)

In [ ]:
%matplotlib inline
_ = interact(plot_2D_GD_interactive, 
            method = fixed('gd'),
             x = fixed(x), 
             y = fixed(y), 
             x_min = fixed(1), 
             y_min = fixed(1), 
             fun = fixed(ackley), 
             x_0 = FloatSlider(min=-4, max=4, step=0.1, value=-2),
             y_0 = FloatSlider(min=-4, max=4, step=0.1, value=-3),
             eta = FloatSlider(min = 0, max = 1, step = 0.01, value=0.2), 
             num_epochs = IntSlider(min = 1, max = 100, step = 1, value=10))

### Momentum


In [ ]:
V = torch.zeros(2)
beta = 0.6
mom = momentum(x_0,y_0,V,beta,learning_rate,num_epochs,ackley)
plot_3D_GD(x,y,mom[:,0],mom[:,1],ackley)

In [ ]:
%matplotlib inline
_ = interact(plot_2D_GD_interactive, 
            method = fixed('momentum'),
             x = fixed(x), 
             y = fixed(y), 
             x_min = fixed(1), 
             y_min = fixed(1), 
             fun = fixed(ackley), 
             x_0 = FloatSlider(min=-4, max=4, step=0.1, value=-2),
             y_0 = FloatSlider(min=-4, max=4, step=0.1, value=-3),
             eta = FloatSlider(min = 0, max = 1, step = 0.01, value=0.2), 
             num_epochs = IntSlider(min = 1, max = 100, step = 1, value=10))

### Nesterov

In [ ]:
nes = Nesterov(x_0,y_0,V,beta,learning_rate,num_epochs,ackley)
plot_3D_GD(x,y,nes[:,0],nes[:,1],ackley)

In [ ]:
%matplotlib inline
_ = interact(plot_2D_GD_interactive, 
            method = fixed('nesterov'),
             x = fixed(x), 
             y = fixed(y), 
             x_min = fixed(1), 
             y_min = fixed(1), 
             fun = fixed(ackley), 
             x_0 = FloatSlider(min=-4, max=4, step=0.1, value=-2),
             y_0 = FloatSlider(min=-4, max=4, step=0.1, value=-3),
             eta = FloatSlider(min = 0, max = 1, step = 0.01, value=0.2), 
             num_epochs = IntSlider(min = 1, max = 100, step = 1, value=10))

In [ ]:
min_glob = torch.Tensor([0,0])
dist_err_gd = err(gd,min_glob,ackley)
dist_err_mom = err(mom,min_glob,ackley)
dist_err_nes = err(nes,min_glob,ackley)

N = range(num_epochs)
plt.semilogy(N,dist_err_gd, marker = 'o', label = 'GD')
plt.semilogy(N,dist_err_mom,marker = 'x', label = 'Momentum')
plt.semilogy(N,dist_err_nes,marker = 'd', label = 'Nesterov')

plt.grid()
plt.xlabel('Epochs')
plt.title('Distance from the global min')
plt.legend()
plt.show()

### Changing the learning rate

In [ ]:
import numpy as np
def step_LR(lr_0,num_epochs,step,gamma):
    lr = np.zeros(num_epochs)
    lr[0] = lr_0
    assert step>0
    for i in range(1,num_epochs):
        if i%step == 0:
            lr[i] = gamma*lr[i-1]
        else:
            lr[i] = lr[i-1]
    return lr


In [ ]:
learning_rate = 0.2
num_epochs = 30

gd = GD(x_0,y_0,learning_rate,num_epochs,ackley)

plot_3D_GD(x,y,gd[:,0],gd[:,1],ackley)

In [ ]:
step = 5
gamma = 0.55
lr = step_LR(learning_rate,num_epochs,step,gamma)
gd_step = GD(x_0,y_0,lr,num_epochs,ackley)

plot_3D_GD(x,y,gd_step[:,0],gd_step[:,1],ackley)

In [ ]:
N = range(num_epochs)
dist_err_gd = err(gd,min_glob,ackley)
dist_err_gd_step = err(gd_step,min_glob,ackley)

plt.semilogy(N,dist_err_gd_step, marker = 'o', label = 'Step scheduler')
plt.semilogy(N,dist_err_gd, marker = 'x', label = 'Without scheduler')

plt.legend()
plt.xlabel('Epochs')
plt.grid()
plt.show()